# 2. 搭建环境并实现简单的应用

## 2.1 搭建环境

In [1]:
# curl -fsSL https://ollama.com/install.sh | sh
# ollama pull gpt-oss:20b
# venv/bin/pip3 install langchain==0.1.14 langchain-community==0.0.38 langchain-core==0.1.53 langchain-openai==0.1.7

import langchain

In [2]:
langchain.__version__

'0.1.14'

## 2.2 实现问答应用——基于基础模型

In [9]:
from langchain_community.llms import Ollama

llm = Ollama(model="gpt-oss:20b")                      # 如果Ollama在本机，可以这样访问，但是当前环境的Ollama在局域网的另一台主机上(192.168.1.211)
print(llm.predict("AI会对人类文明产生深远的影响吗？"))

In [3]:
import requests
import json

class RemoteOllama:
    def __init__(self, host: str, model: str = "gpt-oss:20b"):
        self.host = host
        self.model = model

    def predict(self, prompt: str) -> str:
        url = f"http://{self.host}:11434/v1/completions"  # Ollama 远程 API
        payload = {"model": self.model, "prompt": prompt, "max_tokens": 200}
        headers = {"Content-Type": "application/json"}
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json().get("completion", "")

# 使用示例
llm = RemoteOllama(host="192.168.1.211", model="gpt-oss:20b")
result = llm.predict("AI会对人类文明产生深远的影响吗？")
print(result)

In [24]:
llm = RemoteOllama(host="192.168.1.211", model="gpt-oss:20b")
# print(llm.generate(["AI会对人类文明产生深远的影响吗？"]).generations[0][0].text)
# print(llm.predict("AI会对人类文明产生深远的影响吗？"))
result = llm("AI会对人类文明产生深远的影响吗？")
print(result)

## 2.3 实现翻译应用——基于聊天模型和指令模板

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 创建一个ChatOllama对象
chat = ChatOllama(model="qwen:1.8b")
# 配置聊天模板信息
template = "你是一个翻译助理，请将用户输入的内容由{input_language}直接翻译为{output_language}."  # 设置翻译任务的基本指令
system_message_prompt = SystemMessagePromptTemplate.from_template(template)                     # 基于template创建系统消息指令，向聊天模型明确翻译任务要求
human_template = "{text}"                                                                       # 简单定义人类校址指令模板，仅包含待翻译文本
human_massage_prompt = HumanMessagePromptTemplate.from_template(human_template)                 # 定义人类消息指令
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_massage_prompt])   # 将系统消息指令和人类消息指令组合为完整的聊天指令，用于与聊天模型交互

# 输入信息并获取翻译结果
print(chat.invoke(chat_prompt.format_prompt(input_language="英语",
                                            output_language="中文",
                                            text="Artificial Intelligence (AI) will have a profound impact on human civilization in the coming years.").to_messages()))

/tmp/ipykernel_26242/3516977631.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat = ChatOllama(model="qwen:1.8b")


content='人工智能（AI）将在未来几年对人类文明产生深远影响。' additional_kwargs={} response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-17T03:43:09.24743409Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4081473876, 'load_duration': 24843276, 'prompt_eval_count': 49, 'prompt_eval_duration': 575000000, 'eval_count': 15, 'eval_duration': 3477000000} id='run-a656c02b-4888-434a-8aa4-9603bc8197c2-0'
